In [1]:
import anyio
import openai
from utils.secret_key import OPENAI_KEY


client = openai.AsyncOpenAI(api_key=OPENAI_KEY)

## Default gpt async
async def async_gpt(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = await client.chat.completions.create(
        model="gpt-4",
        messages=messages,
    )
    return response.choices[0].message.content

In [3]:
# Let's see if we can use the async gpt w/ the non async assistant. 
from utils.gpt_code_assistant import generate_from_code_assistant


# Test of main that calls async gpt once, code assistant once, and then async again
async def test_main(assistant, thread, number):
    gpt_result = await async_gpt(f"Return {number}")
    print(f'1/3 ({number})')
    assistant_result = generate_from_code_assistant("Helpful agent", f"return '{number}' (code)", assistant, thread, '')
    print(f'2/3 ({number})')
    gpt_result2 = await async_gpt(f"Return {number}")
    print(f'3/3 ({number})')

    return gpt_result, assistant_result, gpt_result2

In [4]:
#### Testing
from utils.create_assistant import create_agents_and_thread

coding_assistant, thread = create_agents_and_thread()

async def main():
    results = []

    # Limit the number of concurrent tasks w/ semaphore to avoid rate limits
    max_concurrent_tasks = 5  
    semaphore = anyio.Semaphore(max_concurrent_tasks)

    # Async wrapper - modifies results to get the results out of the async
    async def run_and_collect(number):
        async with semaphore: # Limit concurrent tasks
            result = await test_main(coding_assistant, thread, number)
            results.append(result)

    async with anyio.create_task_group() as tg:
        for i in range(3):
            tg.start_soon(run_and_collect, i)
    
    print("All answers collected:")
    for answer in results:
        print(f"{answer}\n")

await main()

1/3 (2)
2/3 (2)
1/3 (0)
2/3 (0)
1/3 (1)
2/3 (1)
3/3 (0)
3/3 (1)
3/3 (2)
All answers collected:
('0', [{'role': 'assistant', 'content': "Certainly! To create a function that returns the string `'0'`, you can follow a similar process as before. Here's the code to achieve that:\n\n```python\ndef return_zero():\n    return '0'\n\n# Calling the function to see the output\nresult = return_zero()\nprint(result)\n```\n\nLet's run this code to ensure it works correctly."}, {'role': 'assistant', 'content': "The function `return_zero()` correctly returns the string `'0'` when called. Let me know if there's anything else you'd like to do!"}], '0')

('1', [{'role': 'assistant', 'content': "Sure! Here's a function that returns the string `'1'`:\n\n```python\ndef return_one():\n    return '1'\n\n# Calling the function to see the output\nresult = return_one()\nprint(result)\n```\n\nI will run this code to demonstrate."}, {'role': 'assistant', 'content': "The function `return_one()` correctly returns t

Based on the above, I believe it's working fine. The log seems to show that everything waits for the code assistant (2nd step) to be finished, which it should since its blocking.

Let's try it..

In [ ]:
#### Async assistant

# Async call code assistant
async def agenerate_from_code_assistant(persona, content, assistant, thread):
    # Delete all messages but the prompt/chronologically first one - this will retain
    thread_messages = client.beta.threads.messages.list(thread.id)
    message_list = [message.model_dump() for message in thread_messages.data]
    for message in message_list[:-1]:
        id = message.get("id")
        await client.beta.threads.messages.delete(
            message_id=id,
            thread_id=thread.id,
        )

    # Add persona and prompt
    await client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",  # I want this to be system but it's only assistant or user. Could try putting it in assistant terms, e..g "I am a judge"..
        content=persona,
    )
    await client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content,
    )

    # Start a new run - this is where we link the thread and the assistant
    run = await client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # TODO: change this to handle streaming eventually, to stop blocking.. will have to see how code interp sessions work w/ it
    while run.status in ["queued", "in_progress"]:
        run = await client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    if run.status == "completed":
        # Get all messages and turn into obj format
        thread_messages = await client.beta.threads.messages.list(
            thread_id=thread.id, order="asc"
        )
        messages = []
        for message in thread_messages.data:
            content = ""
            for content_item in message.content:
                if hasattr(content_item, "text"):
                    content += content_item.text.value
            messages.append({"role": message.role, "content": content})

    # Dunno if this is needed
    else:
        print(f"weird run status: {run.status}")
        messages = []

    # ignore first 3 since they are dummy, persona, and prompt
    return messages[3:]

## Agent creator async
async def acreate_agents_and_thread():
    coding_assistant = await client.beta.assistants.create(
        model="gpt-4o",  # Let's try making this 4o. It should be much cheaper
        instructions="""You are a coding assistant. You *must* use the code interpreter to help solve the question""",
        name="Expert Coder",
        tools=[{"type": "code_interpreter"}],
    )

    coding_thread = await client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": "Take on the given role and answer the users question step by step",
            },
        ],
    )

    return (coding_assistant, coding_thread)